In [18]:
import angr
import logging
l = logging.getLogger()
l.setLevel(logging.ERROR)
proj = angr.Project("../nginx/build/sbin/nginx",load_options={"auto_load_libs":False})

loophead = proj.loader.find_symbol("ngx_event_accept").rebased_addr
entry = proj.factory.entry_state(concrete_fs=True, add_options=angr.options.unicorn)
entry.options.add(angr.options.UNICORN_HANDLE_CGC_TRANSMIT_SYSCALL)
entry.options.update({angr.options.UNICORN_AGGRESSIVE_CONCRETIZATION, angr.options.UNICORN_HANDLE_SYMBOLIC_CONDITIONS, angr.options.UNICORN_HANDLE_CGC_RECEIVE_SYSCALL, angr.options.UNICORN_HANDLE_SYMBOLIC_SYSCALLS})

In [19]:
from detectors import NginxHeartBleedDetector
import sims
det = NginxHeartBleedDetector()
det.attach(proj)

In [29]:
proj.hook_symbol("clock_gettime", sims.gettime_hook())
proj.hook_symbol("ngx_time_init", sims.ngx_time_init_hook())
#proj.hook_symbol("ERR_load_strings", sims.ngx_time_init_hook())

4848325

In [21]:
from main import run_with_timeout

In [30]:
sim = proj.factory.simgr(entry)


In [23]:
#logging.getLogger('angr.engines.unicorn_engine').setLevel('DEBUG'); 
#logging.getLogger('angr.state_plugins.unicorn_engine').setLevel('DEBUG')

In [31]:
def a():
    while True:
        try:
            sim.step()
        except KeyboardInterrupt:
            break

In [32]:
run_with_timeout(a, 1 * 60)

(60, None)

In [33]:
sim

<SimulationManager with 1 active>

main

